<a href="https://colab.research.google.com/github/aleks-haksly/Postypashka/blob/main/DA/03%20HW%20SQL/HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sqlalchemy import text
from sqlalchemy import create_engine
import pandas as pd
from google.colab import userdata

In [2]:
engine = create_engine(userdata.get('supabase'), client_encoding='utf8')

In [3]:
def select(sql):
    sql = text(sql)
    return pd.read_sql(sql, engine)

## Задача 1. Для таблиц audiobooks, audio_cards, и listenings

1. вывести сколько пользователей добавили книгу 'Coraline', сколько пользователей прослушало больше 10%.

2. по каждой операционной системе и тайтлу книги вывести кол-во пользователей, сумму прослушивания в часах. не учитывать тестовые прослушивания.

3. найти книгу, которую слушает больше всего людей.

4. найти книгу, которую чаще всего дослушивают до конца.

5. найти среднее время между первым и вторым прослушиванием у пользователей. удалять тестовые чтения не нужно.

6. для каждого тайтла книги вывести трех пользователей с минимальным временем между первым и последним прослушиваниями книги. удалять тестовые чтения нужно. Отсортировать по книге и разнице времени.

In [26]:
# Задание 1
sql = """
--EXPLAIN ANALYZE
WITH t1 AS (
SELECT
	a.uuid,
	a.duration
FROM
	postupashka.audiobooks a
WHERE
	a.title = 'Coraline'
)

SELECT
	count(DISTINCT l.user_id) AS add_count,
	count(DISTINCT l.user_id) FILTER(WHERE l.position_to * 10 > (SELECT t1.duration FROM t1)) AS pct_10_count
FROM
	postupashka.listenings l
WHERE
	l.audiobook_uuid = (SELECT t1.uuid FROM t1)

"""
select(sql)

,add_count,pct_10_count
0,309,49


In [28]:
# Задание 2 (Иcпользуем группировку до джоина, для ускорения ~ -30%)
sql = """
--EXPLAIN ANALYZE
WITH listening_aggregates AS (
    SELECT
        audiobook_uuid,
        os_name,
        COUNT(DISTINCT user_id) AS user_cnt,
        SUM(position_to - position_from) AS listen_duration
    FROM postupashka.listenings
    WHERE is_test = 0
    GROUP BY audiobook_uuid, os_name
)
SELECT
    a.title,
    l.os_name,
    l.user_cnt,
    l.listen_duration
FROM listening_aggregates l
JOIN postupashka.audiobooks a ON a.uuid = l.audiobook_uuid
ORDER BY a.title, l.os_name;
"""
select(sql)

,title,os_name,user_cnt,listen_duration
0,A Streetcar Named Desire,Android,53,103686
1,A Streetcar Named Desire,iOS,200,497556
2,Call Me By Your Name,Android,70,7383
3,Call Me By Your Name,iOS,284,17592
4,Coraline,Android,98,141049
5,Coraline,iOS,213,288672
6,Missing 411 with David Paulides,Android,137,323688
7,Missing 411 with David Paulides,iOS,120,251945
8,Sapiens. Краткая история человечества,Android,42,506588
9,Sapiens. Краткая история человечества,iOS,200,3848223


In [32]:
# Задание 4
sql = """
SELECT a.title FROM (
SELECT
	l.audiobook_uuid,
	RANK () OVER (ORDER BY	count(DISTINCT l.user_id) desc) AS rnk
FROM postupashka.listenings l
GROUP BY l.audiobook_uuid
) AS top
INNER JOIN postupashka.audiobooks a ON a.uuid = top.audiobook_uuid
WHERE top.rnk = 1
"""
select(sql)

,title
0,The Subtle Art of Not Giving a F*ck


In [33]:
# Задание 5
sql = """
SELECT
	a.title,
	count(l.user_id) AS cnt
FROM
	postupashka.listenings l
LEFT JOIN postupashka.audiobooks a
ON
	l.audiobook_uuid = a."uuid"
WHERE
	l.position_to >= a.duration
GROUP BY
	a.title
ORDER BY
	cnt DESC
LIMIT 1
"""
select(sql)


,title,cnt
0,The Subtle Art of Not Giving a F*ck,68
